In [1]:
import fiona
import geopandas as gpd
import pandas as pd
import numpy as np
import time

today = time.strftime('%Y%m%d')

## p10 parcel-coc tagging

In [2]:
f = (r'C:\Users\ywang\Documents\ArcGIS\Projects\EIR_zoningmods\EIR_zoningmods.gdb')
layers = fiona.listlayers(f)
display(layers)

df =  gpd.read_file(f,
                     driver='FileGDB',
                     layer='p10_pba50_coc_tbl_v2_02172021')
print('read {} rows from parcel_coc tagging file'.format(df.shape[0]))

p10_pba50_cocid = df[['PARCEL_ID', 'coc_id']]
p10_pba50_cocid.loc[p10_pba50_cocid.coc_id != 'CoC', 'coc_id'] = 'NA'

print('{} parcels fall into CoC'.format(p10_pba50_cocid.loc[p10_pba50_cocid.coc_id == 'CoC'].shape[0]))

# export 
p10_pba50_cocid.to_csv('C:\\Users\\ywang\\Box\\Modeling and Surveys\\Urban Modeling\\Bay Area UrbanSim\\PBA50\\Policies\\Zoning Modifications\\p10_pba50_cocid.csv', index=False)

['CoC_only_2020_acs2018',
 'p10_pba50_CoC',
 'p10_pba50_CoC_2020_acs2018',
 'p10_pba50_CoC_2020_acs2018_tbl',
 'p10_pba50_coc_tbl_v1_02122021',
 'p10_pba50_coc_tbl_v2_02172021']

read 1956208 rows from parcel_coc tagging file


C:\Users\ywang\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


299664 parcels fall into CoC


## Create p10_pba50_EIR_attr

In [3]:
p10_pba50_FBP_attr = pd.read_csv(r'C:\Users\ywang\Box\Modeling and Surveys\Urban Modeling\Bay Area UrbanSim\PBA50\Policies\Zoning Modifications\p10_pba50_FBP_attr_20201110.csv')

print('p10_pba50_FBP_attr has {} rows, {} unique PARCEL_IDs'.format(p10_pba50_FBP_attr.shape[0],
                                                                    len(p10_pba50_FBP_attr.PARCEL_ID.unique())))
print('p10_pba50_FBP_attr fields: {}'.format(list(p10_pba50_FBP_attr)))

C:\Users\ywang\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


p10_pba50_FBP_attr has 1956208 rows, 1956208 unique PARCEL_IDs
p10_pba50_FBP_attr fields: ['OBJECTID', 'PARCEL_ID', 'DEVELOPMEN', 'LAND_VALUE', 'ACRES', 'COUNTY_ID', 'ZONE_ID', 'PROPORTION', 'TAX_EXEMPT', 'APN', 'GEOM_ID', 'IMPUTATION', 'ID', 'CENTROID', 'X', 'Y', 'geom_id_s', 'manual_cou', 'jurisdicti', 'pda_id', 'tpp_id', 'exp_id', 'exp_score', 'opp_id', 'zoningmodc', 'perffoot', 'perfarea', 'mapshell', 'tpa_id', 'perfarea2', 'alt_zoning', 'zonetype', 'pubopp_id', 'puboppuse', 'juris_id', 'hra_id', 'trich_id', 'cat_id', 'chcat', 'zoninghzca', 'gg_id', 'tra_id', 'sesit_id', 'ppa_id', 'exp2020_id', 'pba50chcat', 'exsfd_id', 'chcatwsfd', 'pba50zonin', 'nodev', 'juris', 'fbp_gg_id', 'fbp_tra_id', 'fbp_sesit_', 'fbp_ppa_id', 'fbp_exp202', 'fbpchcat', 'fbp_exsfd_', 'fbpchcatws', 'fbpzoningm', 'Shape_Leng', 'Shape_Area']


In [4]:
# create "p10_pba50_EIR_attr", starting with copying "p10_pba50_FBP_attr" 
p10_pba50_EIR_attr = p10_pba50_FBP_attr.copy()


# create columns for EIR variables, starting with copying values from correponding FBP fields

fbp_cols = ['fbp_gg_id', 'fbp_tra_id', 'fbp_sesit_',
            'fbp_ppa_id', 'fbp_exp202', 'fbp_exsfd_']

for colname_fbp in fbp_cols:
    colname_eir = colname_fbp.replace('fbp','eir')
    p10_pba50_EIR_attr[colname_eir] = p10_pba50_EIR_attr[colname_fbp]


# merge "coc_id" field to "p10_pba50_EIR_attr"
p10_pba50_EIR_attr['PARCEL_ID'] = p10_pba50_EIR_attr['PARCEL_ID'].apply(lambda x: int(round(x)))
p10_pba50_cocid['PARCEL_ID'] = p10_pba50_cocid['PARCEL_ID'].apply(lambda x: int(round(x)))

p10_pba50_EIR_attr = p10_pba50_EIR_attr.merge(p10_pba50_cocid, on='PARCEL_ID', how='outer')
p10_pba50_EIR_attr.rename(columns={'coc_id':'eir_coc_id'}, inplace=True)
print('p10_pba50_EIR_attr has {} rows, {} unique PARCEL_IDs'.format(p10_pba50_EIR_attr.shape[0],
                                                                    len(p10_pba50_EIR_attr.PARCEL_ID.unique())))

C:\Users\ywang\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


p10_pba50_EIR_attr has 1956208 rows, 1956208 unique PARCEL_IDs


In [5]:
# add "eirzoningm" field using the new concatenation

eir_zmodcat_cols = ['juris','eir_gg_id', 'eir_tra_id', 'eir_sesit_', 'eir_coc_id', 'eir_ppa_id', 'eir_exp202', 'eir_exsfd_']

p10_pba50_EIR_attr[eir_zmodcat_cols] = p10_pba50_EIR_attr[eir_zmodcat_cols].apply(lambda x: x.fillna('NA'))
p10_pba50_EIR_attr['eirzoningm'] = p10_pba50_EIR_attr[eir_zmodcat_cols].apply(lambda row: ''.join(row.values.astype(str)), axis=1)
print('{} unique eirzoningmod:'.format(len(p10_pba50_EIR_attr.eirzoningm.unique())))
display(p10_pba50_EIR_attr.eirzoningm.unique())

print('p10_pba50_EIR_attr has the following fields: {}'.format(list(p10_pba50_EIR_attr)))

3504 unique eirzoningmod:


array(['livermoreNANAHRADISNANAinNA', 'livermoreNAtra3DISNANAinNA',
       'haywardNANANANANAinNA', ..., 'campbellNAtra2cHRANANAinNA',
       'napaNANANANANAexp1sfd', 'unincorporated_marinNANADISNANAubz_npNA'],
      dtype=object)

p10_pba50_EIR_attr has the following fields: ['OBJECTID', 'PARCEL_ID', 'DEVELOPMEN', 'LAND_VALUE', 'ACRES', 'COUNTY_ID', 'ZONE_ID', 'PROPORTION', 'TAX_EXEMPT', 'APN', 'GEOM_ID', 'IMPUTATION', 'ID', 'CENTROID', 'X', 'Y', 'geom_id_s', 'manual_cou', 'jurisdicti', 'pda_id', 'tpp_id', 'exp_id', 'exp_score', 'opp_id', 'zoningmodc', 'perffoot', 'perfarea', 'mapshell', 'tpa_id', 'perfarea2', 'alt_zoning', 'zonetype', 'pubopp_id', 'puboppuse', 'juris_id', 'hra_id', 'trich_id', 'cat_id', 'chcat', 'zoninghzca', 'gg_id', 'tra_id', 'sesit_id', 'ppa_id', 'exp2020_id', 'pba50chcat', 'exsfd_id', 'chcatwsfd', 'pba50zonin', 'nodev', 'juris', 'fbp_gg_id', 'fbp_tra_id', 'fbp_sesit_', 'fbp_ppa_id', 'fbp_exp202', 'fbpchcat', 'fbp_exsfd_', 'fbpchcatws', 'fbpzoningm', 'Shape_Leng', 'Shape_Area', 'eir_gg_id', 'eir_tra_id', 'eir_sesit_', 'eir_ppa_id', 'eir_exp202', 'eir_exsfd_', 'eir_coc_id', 'eirzoningm']


In [6]:
# export
p10_pba50_EIR_attr.to_csv('C:\\Users\\ywang\\Box\\Modeling and Surveys\\Urban Modeling\\Bay Area UrbanSim\\PBA50\\Policies\\Zoning Modifications\\p10_pba50_EIR_attr_{}.csv'.format(today), index=False)

## create zoning_mods lookup table

In [7]:
# bring in zoningmods fields from FBP as place holders

# read these fields in s24
lookup_fbp = pd.read_csv(r'C:\Users\ywang\Documents\GitHub\bayarea_urbansim\data\zoning_mods_24.csv',
                         usecols = ['fbpzoningmodcat', 'add_bldg', 'drop_bldg', 'dua_up', 'far_up', 
                                    'dua_down', 'far_down', 'subsidy', 'notes', 'res_rent_cat', 'job_out_cat'])

print('zoning_mods_24 has {} unique fbpzoningmodcat'.format(lookup_fbp.shape[0]))
display(lookup_fbp.head())
#print(list(lookup_fbp))
print('dua_up has the following values: {}'.format(list(lookup_fbp.dua_up.unique())))
print('add_bldg has the following values: {}'.format(list(lookup_fbp.add_bldg.unique())))

p10_pba50_EIR_attr_modcat = p10_pba50_EIR_attr.merge(lookup_fbp, 
                                                     left_on='fbpzoningm',
                                                     right_on='fbpzoningmodcat', how='left')
print('p10_pba50_EIR_attr_modcat has {} rows'.format(p10_pba50_EIR_attr_modcat.shape[0]))

zoning_mods_24 has 2970 unique fbpzoningmodcat


,fbpzoningmodcat,add_bldg,drop_bldg,dua_up,far_up,dua_down,far_down,subsidy,notes,res_rent_cat,job_out_cat
0,alamedaGGNADISNAinNA,NaN,NaN,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,alamedaGGNADISNAinsfd,NaN,NaN,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,alamedaGGNAHRADISNAinNA,HM,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,alamedaGGNAHRADISNAinsfd,HM,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,alamedaGGNAHRANAinNA,HM,NaN,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


dua_up has the following values: [25.0, 35.0, 75.0, 50.0, nan, 100.0, 200.0, 150.0, 2.75, 1.5]
add_bldg has the following values: [nan, 'HM', 'IW', 'HS']
p10_pba50_EIR_attr_modcat has 1956208 rows


In [8]:
# collapsed to lookup table based on 'eirzoningm' and EIR geography fields, 'fbpzoningmodcat'
#      was kept to get values for far_up, dua_down, etc. 

p10_pba50_EIR_modcat = p10_pba50_EIR_attr_modcat[['ACRES', 'fbpzoningmodcat', 'eirzoningm', 'juris',
                                                  'eir_gg_id', 'eir_tra_id', 'eir_sesit_', 'eir_coc_id', 
                                                  'eir_ppa_id', 'eir_exp202', 'eir_exsfd_',
                                                  'add_bldg', 'drop_bldg', 'dua_up', 'far_up', 
                                                  'dua_down', 'far_down', 'subsidy', 
                                                  'notes', 'res_rent_cat', 'job_out_cat']]

p10_pba50_EIR_modcat_df = p10_pba50_EIR_modcat[['fbpzoningmodcat','eirzoningm', 'juris',
                                                'eir_gg_id', 'eir_tra_id', 'eir_sesit_', 'eir_coc_id', 
                                                'eir_ppa_id', 'eir_exp202', 'eir_exsfd_',
                                                'add_bldg', 'drop_bldg', 'dua_up', 'far_up', 
                                                'dua_down', 'far_down', 'subsidy', 
                                                'notes', 'res_rent_cat', 'job_out_cat']].drop_duplicates()

# rename columns
p10_pba50_EIR_modcat_df.rename(columns = {'eir_gg_id': 'gg_id',
                                          'eir_tra_id': 'tra_id',
                                          'eir_sesit_': 'sesit_id',
                                          'eir_coc_id': 'coc_id',
                                          'eir_ppa_id': 'ppa_id',
                                          'eir_exp202': 'exp2020_id',
                                          'eir_exsfd_': 'exsfd_id'}, inplace=True)

# add 'FREQUENCE', 'SUM_ACRES' columns
p10_pba50_EIR_stats = p10_pba50_EIR_modcat.groupby('eirzoningm').agg({'ACRES': ['count','sum']}).reset_index()
p10_pba50_EIR_stats.columns = ['eirzoningm', 'FREQUENCY', 'SUM_ACRES']
print('p10_pba50_EIR_stats has {} rows'.format(p10_pba50_EIR_stats.shape[0]))

p10_pba50_EIR_modcat_df = p10_pba50_EIR_modcat_df.merge(p10_pba50_EIR_stats, on='eirzoningm', how='left')
print('p10_pba50_EIR_modcat_df has {} rows'.format(p10_pba50_EIR_modcat_df.shape[0]))

p10_pba50_EIR_stats has 3504 rows
p10_pba50_EIR_modcat_df has 3504 rows


In [9]:
# add 'manual_county' column
juris_county = pd.read_csv(r'C:\Users\ywang\Documents\GitHub\petrale\zones\jurisdictions\juris_county_id.csv',
                           usecols = ['juris_name_full', 'county_id'])
juris_county.columns = ['juris','manual_county']

p10_pba50_EIR_modcat_df = p10_pba50_EIR_modcat_df.merge(juris_county, on='juris', how='left')

In [10]:
# Update "dua_up" and "add_bldg" for residential - EIR Alt1

df1 = p10_pba50_EIR_modcat_df.copy()

df1.dua_up = np.nan
df1.loc[(df1.add_bldg == 'HM') | (df1.add_bldg == 'HS'), 'add_bldg'] = np.nan

df1.loc[(df1.gg_id=='GG') & (df1.tra_id=='tra1'), 'dua_up'] = 300
df1.loc[(df1.gg_id=='GG') & (df1.tra_id=='tra1'), 'add_bldg'] = 'HM'

df1.loc[(df1.gg_id=='GG') & (df1.tra_id=='tra2a'), 'dua_up'] = 250
df1.loc[(df1.gg_id=='GG') & (df1.tra_id=='tra2a'), 'add_bldg'] = 'HM'

df1.loc[(df1.gg_id=='GG') & (df1.tra_id=='tra2b'), 'dua_up'] = 200
df1.loc[(df1.gg_id=='GG') & (df1.tra_id=='tra2b'), 'add_bldg'] = 'HM'

df1.loc[(df1.gg_id=='GG') & (df1.tra_id=='tra2c'), 'dua_up'] = 150
df1.loc[(df1.gg_id=='GG') & (df1.tra_id=='tra2c'), 'add_bldg'] = 'HM'

df1.loc[(df1.gg_id=='GG') & (df1.tra_id=='tra3'), 'dua_up'] = 100
df1.loc[(df1.gg_id=='GG') & (df1.tra_id=='tra3'), 'add_bldg'] = 'HM'

# Dave suggested not upzone non-TRA areas
# df1.loc[(df1.gg_id=='GG') & (df1.tra_id=='NA'), 'dua_up'] = 50
# df1.loc[(df1.gg_id=='GG') & (df1.tra_id=='NA'), 'add_bldg'] = 'HM'

In [11]:
# Update nonresidential "add_bldg" for PPAs
df1.loc[(df1.gg_id=='GG') & (df1.ppa_id=='ppa') & (df1.tra_id != 'tra1'), 'add_bldg'] = 'IW'

# Oakland airport
df1.loc[(df1.eirzoningm == 'oaklandNAtra3NANAppainNA') | (df1.eirzoningm == 'oaklandNAtra3NACoCppainNA'), 'add_bldg'] = 'IW'

In [12]:
# add 'modcat_id' column and recoder the fields
df1['modcat_id'] = df1.index + 1
df1 = df1[['eirzoningm', 'modcat_id', 'FREQUENCY', 'SUM_ACRES', 'manual_county', 'juris', 
           'gg_id', 'tra_id', 'sesit_id', 'coc_id', 'ppa_id', 'exp2020_id', 'exsfd_id', 
           'add_bldg', 'drop_bldg', 'dua_up', 'far_up', 'dua_down', 'far_down', 'subsidy', 'notes', 'res_rent_cat', 'job_out_cat']]

In [13]:
# export
df1.rename(columns={'eirzoningm': 'eirzoningmodcat'}, inplace=True)
print('export zoning_mods lookup table of {} rows'.format(df1.shape[0]))
df1.to_csv('C:\\Users\\ywang\\Box\\Modeling and Surveys\\Urban Modeling\\Bay Area UrbanSim\\PBA50\\Policies\\Zoning Modifications\\BAUS input files\\zoning_mods_26_{}.csv'.format(today), index=False)

export zoning_mods lookup table of 3504 rows


In [14]:
df1

,eirzoningmodcat,modcat_id,FREQUENCY,SUM_ACRES,manual_county,juris,gg_id,tra_id,sesit_id,coc_id,...,add_bldg,drop_bldg,dua_up,far_up,dua_down,far_down,subsidy,notes,res_rent_cat,job_out_cat
0,livermoreNANAHRADISNANAinNA,1,462,501.766781,1,livermore,NA,NA,HRADIS,NA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,livermoreNAtra3DISNANAinNA,2,3041,1998.371566,1,livermore,NA,tra3,DIS,NA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,haywardNANANANANAinNA,3,1027,1180.939254,1,hayward,NA,NA,NA,NA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,unincorporated_sonomaNANAHRADISNANAoutNA,4,2947,140734.550811,97,unincorporated_sonoma,NA,NA,HRADIS,NA,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,fremontNANAHRADISNANAinNA,5,6692,2717.938841,1,fremont,NA,NA,HRADIS,NA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3499,unincorporated_santa_claraNANAHRADISNANAexp3_a...,3500,1,2.450437,85,unincorporated_santa_clara,NA,NA,HRADIS,NA,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3500,cloverdaleNANANANANAoutNA,3501,1,1.071807,97,cloverdale,NA,NA,NA,NA,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
3501,campbellNAtra2cHRANANAinNA,3502,1,2.044626,85,campbell,NA,tra2c,HRA,NA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3502,napaNANANANANAexp1sfd,3503,1,20.309329,55,napa,NA,NA,NA,NA,...,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
